# 引言

假设检验包括显著性检验和其他检验，我们今天讲显著性检验，

按照类型：显著性检验包括参数检验（对数据要求高，结果很可信）和非参数检验（对数据没要求，结果不如参数检验可信）。

按照对象情况：显著性检验包括双样本检验（t检验）、多样本检验（f检验）和其他检验

今天要用到的库是pingouin

***学习显著性检验的代码实现可以不深究原理，而是搞清楚这几个东西：***

- 每个方法的使用场景和目的
- 每个方法的假设与验证假设的方法
- 怎么看结果

任何假设检验都会求出一个统计量和p值，统计量无所谓（但是要写在论文里作为结果呈现），这个p值就是显著性检验最重要的结果，p<0.05则有显著差异，p>0.05则没有显著差异

显著性检验一般结果要在论文里呈现的是统计量和p值（如果方便可以加一个cohen-d）

# 双样本显著检验——T检验及其非参数检验
<br><br>

### 1. 独立样本 t 检验（Independent Samples t-Test）

**目的和用处：**
- **目的：** 检验两个独立的样本（如两组不同的参与者、两种不同的治疗方法等）的平均值是否有显著差异。
- **用处：** 用于确定两组独立样本的均值是否有显著差异，从而评估两组之间的差异性。

**模型假设：**
1. **独立性假设：** 两个样本是相互独立的。
2. **正态性假设：** 每个样本的观察值来自于正态分布。
3. **方差齐性假设：** 两个样本的总体方差应该相等。


In [1]:
import pingouin as pg

# 独立样本
group_A = [5.5, 2.4, 6.8, 9.6, 4.2]
group_B = [6.4, 3.4, 6.4, 11., 4.8]

In [2]:
# 正态性检验
print(pg.normality(group_A))
print(pg.normality(group_B))

# 方差齐性检验
print(pg.homoscedasticity([group_A, group_B]))

          W      pval  normal
0  0.989862  0.979208    True
          W      pval  normal
0  0.903597  0.430084    True
               W      pval  equal_var
levene  0.010916  0.919362       True


In [3]:
pg.ttest(x=group_A, 
        y=group_B, 
        correction=False,  # 该参数在方差齐性通过时为False，不通过时为True
        alternative='two-sided',   # 双边检验：two-sided, greater, less
        )

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.396551,8,two-sided,0.702066,"[-4.77, 3.37]",0.250801,0.517,0.064257



### 2. 配对样本 t 检验（Paired Samples t-Test）

**目的和用处：**
- **目的：** 检验同一组体系中的两个相关样本（如同一组人群在不同时间点或条件下的观察值）的平均值是否有显著差异。
- **用处：** 用于确定两个相关样本的平均值是否有显著差异，从而评估在不同条件下的变化效果。

**模型假设：**
1. **相关性假设：** 两个样本是相关的，即两组观察值是配对的。
2. **差值正态性假设：** 差值（即每对样本的差异）来自于正态分布。


In [4]:
import pandas as pd
import pingouin as pg

# 生成示例数据
df = pd.DataFrame({
    'before': [21, 25, 24, 26, 23, 22, 20, 21, 24, 23, 22, 25, 21, 22, 24, 23, 21, 22, 20, 21, 22, 23, 24, 25, 22, 21, 20],
    'after': [23, 27, 26, 28, 25, 24, 22, 23, 26, 25, 24, 27, 23, 24, 26, 25, 23, 24, 22, 23, 24, 25, 26, 27, 24, 23, 22]
    })
df

,before,after
0,21,23
1,25,27
2,24,26
3,26,28
4,23,25
5,22,24
6,20,22
7,21,23
8,24,26
9,23,25


In [5]:
# 相关性检验
print(pg.corr(df['before'], df['after']))

# 计算差值，差值正态性检验
df['diff'] = df['after'] - df['before']
print(pg.normality(df['diff']))

          n    r       CI95%  p-val BF10  power
pearson  27  1.0  [1.0, 1.0]    0.0  inf      1
        W  pval  normal
diff  1.0   1.0    True


d:\Coding\python\envs\py_ds\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [6]:
# 配对样本 t 检验
pg.ttest(x=df['before'], 
        y=df['after'], 
        paired=True,   # 表示进行配对样本检验
        alternative='two-sided',  # 双边检验
        )

d:\Coding\python\envs\py_ds\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
d:\Coding\python\envs\py_ds\Lib\site-packages\pingouin\parametric.py:280: RuntimeWarning: invalid value encountered in multiply
  ci = np.array([tval - tcrit, tval + tcrit]) * se


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-inf,26,two-sided,0.0,"[nan, nan]",1.179609,nan,0.999958



### 3. 单样本 t 检验（One Sample t-Test）

**目的和用处：**
- **目的：** 检验单个样本的平均值是否与某个已知的总体均值有显著差异。
- **用处：** 用于确定单个样本的平均值是否有显著不同于已知总体均值的水平。

**模型假设：**
1. **正态性假设：** 单个样本的观察值来自于正态分布。
2. **总体均值假设：** 单个样本的均值等于已知的总体均值。



In [7]:
group_A = [5.5, 2.4, 6.8, 9.6, 4.2]

In [8]:
pg.normality(group_A)

,W,pval,normal
0,0.989862,0.979208,True


In [9]:
pg.ttest(x=group_A, 
        y=7,  # 是否认为平均值与y无显著差异
        alternative='two-sided',  # 双边检验：greater，less
        )

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.068593,4,two-sided,0.345452,"[2.32, 9.08]",0.47789,0.603,0.132614


### 4. Mann-Whitney U 检验（独立样本非参数检验）

**目的和用处：**
- **目的：** 检验两个独立的样本的中位数或分布是否有显著差异，而不需要假设数据来自正态分布。
- **用处：** 适用于数据不满足正态分布假设的情况，或者样本大小较小，难以做出正态分布假设的情况。

**模型假设：**
- **独立性假设：** 两个样本是相互独立的。
- **中位数假设（或分布假设）：** 非参数检验通常基于中位数或分布的差异来进行推断，而不是基于均值。


In [10]:
# Mann-Whitney U 检验
pg.mwu(x=group_A, 
    y=group_B, 
    alternative='two-sided',
    )

,U-val,alternative,p-val,RBC,CLES
MWU,11.0,two-sided,0.834035,0.12,0.44





### 5. Wilcoxon 符号秩检验（配对样本非参数检验）

**目的和用处：**
- **目的：** 检验配对样本中的差异（如同一组体系中的两个相关样本）的中位数或分布是否有显著差异。
- **用处：** 适用于配对样本不满足正态分布假设的情况，或者数据形状无法明确假设的情况。

**模型假设：**
- **相关性假设：** 两个样本是相关的，即存在配对关系。
- **差值的中位数假设（或分布假设）：** 非参数检验通常基于配对差值的中位数或分布的差异来进行推断。

In [11]:
pg.wilcoxon(x=df['before'],
        y=df['after'],
        alternative='two-sided',
        )

,W-val,alternative,p-val,RBC,CLES
Wilcoxon,0.0,two-sided,1.490116e-08,-1.0,0.209877
